# **Importing libraries**

In [3]:

from datetime import datetime, date
from sqlalchemy import DATE, TEXT, DATETIME, INTEGER
import os
import pandas as pd
import requests
import json
import numpy as np
import sqlite3

def date_parser(data_str):
    """
        Format any date in YYYY-MM-DD HH:MM:SS
    Args:
        data_str (str): string with the date

    Returns:
        str: string with the date formatted in YYYY-MM-DD HH:MM:SS
    """
    try:
        # Tenta analisar a data de entrada em diferentes formatos
        data_str = str(data_str)
        if data_str[:2] == '00':
            data_str = '20'+str(data_str)[2:]
        # data = datetime(year=int('20'+data_str[2:4]), month=int(data_str[5:7], 10), day=int(data_str[8:10]), hour=int(data_str[11:13]), minute=int(data_str[14:16]), second=int(data_str[17:19]),microsecond=int(0))
        data = datetime.strptime(data_str, '%Y-%m-%d %H:%M:%S')
        if data_str[:2] == '00':
            print(data_str)
            print(data)
    except ValueError:
        try:
            data = datetime.strptime(data_str, '%Y-%m-%d %H:%M:%S')
        except ValueError:
            try:
                data = datetime.strptime(data_str, '%Y-%m-%d')
            except ValueError:
                try:
                    data = datetime.strptime(data_str, '%d/%m/%Y %H:%M:%S')
                except ValueError:
                    try:
                        data = datetime.strptime(data_str, '%d/%m/%Y %H:%M')
                    except ValueError:
                        try:
                            data = datetime.strptime(data_str, '%d/%m/%Y')
                        except ValueError:
                            try:
                                data = datetime.strptime(data_str, '%Y-%m-%d %H:%M')
                            except ValueError:
                                return None  # Retornar None se a data não pôde ser analisada

    # Formate a data no formato desejado
    data_formatada = data.strftime('%Y-%m-%d %H:%M:%S.%f')
    data_obj = datetime.strptime(data_formatada, '%Y-%m-%d %H:%M:%S.%f')
    return data_obj

def trf_dtypes(schema,dataframe):
    """
    Transforma os tipos de dados do dataframe de acordo com o schema especificado.

    Args:
        schema_bq (dict): Dicionário contendo o schema do BigQuery, com os nomes das colunas e os tipos de dados.
        dataframe (pandas.DataFrame): O dataframe a ser transformado.

    Returns:
        dict: Um dicionário contendo as informações do schema e o dataframe transformado.
    """
    # print(dataframe.dtypes)
    de_para = {
            DATETIME: 'DATETIME',
            DATE: 'DATE',
            TEXT: 'TEXT',
            INTEGER: 'INTEGER'
        }
    for key, value in schema.items():
        # print(f'Parsing {key} for type {value}')
        type = de_para.get(value)
        # print(f'Parsing {key} for type {type}')
        match(type):
            case 'INTEGER':
                dataframe[key] = dataframe[key].astype(pd.Int64Dtype())
            case 'DATETIME':
                dataframe[key] = dataframe[key].apply(date_parser)
                # dataframe[key] = dataframe[key].apply(lambda x: x if not pd.isna(x) else None)
                # dataframe[key] = pd.to_datetime(dataframe[key],format='%Y-%m-%d %H:%M:%S')
            case 'DATE':
                # dataframe[key] = dataframe[key].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())
                # dataframe[key] = dataframe[key].apply(lambda x: None if pd.isna(x) else x)
                dataframe[key] = pd.to_datetime(dataframe[key], format='%Y-%m-%d')
            # case 'Float':
            #     dataframe[key] = dataframe[key].apply(trf_numeric)
            #     dataframe[key] = dataframe[key].round(decimals=7).astype(float)
            #     context = decimal.Context(prec=10,Emax=99999, Emin=-99999)
            #     dataframe[key] = dataframe[key].apply(context.create_decimal_from_float)
            case 'TEXT':
                # dataframe[key] = dataframe[key].apply(trf_str)
                dataframe[key] = dataframe[key].replace("null", None)
                dataframe[key] = dataframe[key].astype(pd.StringDtype())
    return dataframe


# **Solution for the first question**

In [2]:
print("\n-----[E] Extracting data of JSON files-----\n")

# Declare the path to the folder containing the JSON files
path = r'source\VRA'

# Declare the variable to store all JSON data
data_json = pd.DataFrame()

schema = {
    'icao_empresa_aerea':TEXT,
    'numero_voo':TEXT,
    'codigo_autorizacao':TEXT,
    'codigo_tipo_linha':TEXT,
    'icao_aerodromo_origem':TEXT,
    'icao_aerodromo_destino':TEXT,
    'partida_prevista':DATETIME,
    'partida_real':DATETIME,
    'chegada_prevista':DATETIME,
    'chegada_real':DATETIME,
    'situacao_voo':TEXT,
    'codigo_justificativa':TEXT}

# Loop through each JSON file in the folder
for file in os.listdir(path):
    with open(os.path.join(path, file), 'r',encoding='utf-8-sig') as f:
        data = json.load(f)
    print(f'Extracted {file} -> {len(data)} rows')
    data_json = pd.concat([data_json, pd.DataFrame(data)], ignore_index=True)
print(f"\nExtracted -> {len(data_json)} rows")

# Normalize the headers for snake case pattern
rename_dict = {
    "ICAOEmpresaAérea":"icao_empresa_aerea",
    "NúmeroVoo":"numero_voo",
    "CódigoAutorização":"codigo_autorizacao",
    "CódigoTipoLinha":"codigo_tipo_linha",
    "ICAOAeródromoOrigem":"icao_aerodromo_origem",
    "ICAOAeródromoDestino":"icao_aerodromo_destino",
    "PartidaPrevista":"partida_prevista",
    "PartidaReal":"partida_real",
    "ChegadaPrevista":"chegada_prevista",
    "ChegadaReal":"chegada_real",
    "SituaçãoVoo":"situacao_voo",
    "CódigoJustificativa":"codigo_justificativa"}

data_json.rename(columns=rename_dict,inplace=True)
print(f'\n{data_json.dtypes}\n')

# Transforming datetime data types
print("-----[T] Transforming the data types-----")
data_json = trf_dtypes(schema,data_json)
print(f'\n{data_json.dtypes}\n')

# Replace NaN values
# data_json = data_json.replace(pd.NA,None)
# data_json = data_json.where(pd.notna(data_json), None)

# Saving the data as a SQLite file
print("-----[L] Loading to the database-----")
data_json.to_sql('vra', con='sqlite:///data\data.db?check_same_thread=False', index=False,if_exists='replace',dtype=schema)



<>:60: SyntaxWarning: invalid escape sequence '\d'
<>:60: SyntaxWarning: invalid escape sequence '\d'
C:\Users\gusta\AppData\Local\Temp\ipykernel_30128\2379615460.py:60: SyntaxWarning: invalid escape sequence '\d'
  data_json.to_sql('vra', con='sqlite:///data\data.db?check_same_thread=False', index=False,if_exists='replace',dtype=schema)



-----[E] Extracting data of JSON files-----

Extracted VRA_20211.json -> 58550 rows
Extracted VRA_202110.json -> 59430 rows
Extracted VRA_202111.json -> 64004 rows
Extracted VRA_20212.json -> 43692 rows
Extracted VRA_20213.json -> 39514 rows
Extracted VRA_20214.json -> 29122 rows
Extracted VRA_20215.json -> 35750 rows
Extracted VRA_20216.json -> 40955 rows
Extracted VRA_20217.json -> 53313 rows
Extracted VRA_20218.json -> 54928 rows
Extracted VRA_20219.json -> 56545 rows

Extracted -> 535803 rows

icao_empresa_aerea        object
numero_voo                object
codigo_autorizacao        object
codigo_tipo_linha         object
icao_aerodromo_origem     object
icao_aerodromo_destino    object
partida_prevista          object
partida_real              object
chegada_prevista          object
chegada_real              object
situacao_voo              object
codigo_justificativa      object
dtype: object

-----[T] Transforming the data types-----

icao_empresa_aerea        string[python]
n

535803

# **Solution for the Second question**

In [3]:
print("\n-----[E] Extracting data of CSV files-----\n")

# Declare the path to the folder containing the JSON files
path = r'source\AIR_CIA'

# Declare the variable to store all JSON data
data_csv = pd.DataFrame()

schema = {
        'razao_social':TEXT,
        'icao_iata':TEXT,
        'cnpj':TEXT,
        'atividades_aereas':TEXT,
        'endereco_sede':TEXT,
        'telefone':TEXT,
        'e_mail':TEXT,
        'decisao_operacional':TEXT,
        'data_decisao_operacional':DATE,
        'validade_operacional':DATE
        }

# Loop through each CSV file in the folder
names = list(schema.keys())
for file in os.listdir(path):
    df = pd.read_csv(os.path.join(path, file), encoding='utf-8', sep=';', names=names,dtype=str,parse_dates=names[8:],dayfirst=True,header=0)
    print(f'Extracted {file} -> {len(df)} rows')
    data_csv = pd.concat([data_csv, df], ignore_index=True)
print(f"\nExtracted -> {len(data_csv)} rows")
print(f'\n{data_csv.dtypes}\n')

# Splitting the icao_iata column
print("-----[T] Transforming data -----")
data_csv[['icao', 'iata']] = data_csv['icao_iata'].str.split(' ', expand=True)

# Removing the icao_iata column
data_csv = data_csv.drop('icao_iata', axis=1)
del schema['icao_iata']

# Adding new columns in the schema dictionary
schema['icao'] = TEXT
schema['iata'] = TEXT

# Transforming data types
data_csv = trf_dtypes(schema,data_csv)
print(f'\n{data_csv.dtypes}\n')

# data_csv.replace({pd.NaT: None}, inplace=True)
# data_csv = data_csv.replace([float('nan')], [None])

# print(data_csv['data_decisao_operacional'])
# print(data_csv['validade_operacional'])

# data_csv = data_csv.where(pd.notna(data_csv), None)

# Saving the data as a SQLite file
print("-----[L] Loading to the database-----")
data_csv.to_sql('air_cia', con='sqlite:///data\data.db?check_same_thread=False', index=False,if_exists='replace',dtype=schema)


-----[E] Extracting data of CSV files-----

Extracted ANAC_20211220_203627.csv -> 13 rows
Extracted ANAC_20211220_203643.csv -> 5 rows
Extracted ANAC_20211220_203733.csv -> 2 rows

Extracted -> 20 rows

razao_social                        object
icao_iata                           object
cnpj                                object
atividades_aereas                   object
endereco_sede                       object
telefone                            object
e_mail                              object
decisao_operacional                 object
data_decisao_operacional    datetime64[ns]
validade_operacional        datetime64[ns]
dtype: object

-----[T] Transforming data -----

razao_social                string[python]
cnpj                        string[python]
atividades_aereas           string[python]
endereco_sede               string[python]
telefone                    string[python]
e_mail                      string[python]
decisao_operacional         string[python]
data_decisao_ope

<>:57: SyntaxWarning: invalid escape sequence '\d'
<>:57: SyntaxWarning: invalid escape sequence '\d'
C:\Users\gusta\AppData\Local\Temp\ipykernel_30128\1056845284.py:57: SyntaxWarning: invalid escape sequence '\d'
  data_csv.to_sql('air_cia', con='sqlite:///data\data.db?check_same_thread=False', index=False,if_exists='replace',dtype=schema)


20

# **Solution for the third question**

In [ ]:
# Define the API endpoint
url = "https://airport-info.p.rapidapi.com/airport"

# Define the headers
headers = {
	"X-RapidAPI-Key": "58b48914f9mshcdae739ac9f0de7p15b71djsnb39bd8f9e064",
	"X-RapidAPI-Host": "airport-info.p.rapidapi.com"
}
# List for storing the data
data = []

# Get the all unique IICAOs
icao_destino = data_json['icao_aerodromo_destino'].unique().tolist()
icao_origem = data_json['icao_aerodromo_origem'].unique().tolist()
# icao_air_cia = data_csv['icao'].unique().tolist()
icao = icao_destino + icao_origem
icao = list(set(icao))
print(f'{len(icao)} ICAOs to search')

# Loop through each ICAO
for i in icao:
    # print(f'Searching {i}')
    # Define the querystring parameters
    querystring = {"icao":i}
    
    # Make the API request
    response = requests.get(url, headers=headers, params=querystring)
    
    # Check if the request was successful
    if response.status_code == 200:
        result = response.json()
        
        try:
            # Verify if the 'error' key is present in the response
            erro = result['error']
            print(f'{i} -> {erro}')
        except:
            # Add the data to the list
            data.append(response.json())
    else:
        print(response.status_code)
print(f'\nExtracted -> {len(data)} rows')

# Save the data to a SQLite database
df = pd.DataFrame(data)
df = df.replace('', None)
df.to_sql('airport', con='sqlite:///data\data.db?check_same_thread=False', index=False,if_exists='replace')


# **Solution for the fourth question**

**First view**

In [6]:
# Conecte-se ao banco de dados SQLite (ou crie um novo)
conn = sqlite3.connect('.\data\data.db')

# Crie um cursor para executar comandos SQL
cursor = conn.cursor()

# Defina o comando SQL para criar uma view
query = """
CREATE VIEW top_air_routes_per_company AS
SELECT 
cia.razao_social AS razao_social, rotas.icao_empresa_aerea, rotas.frequencia, rank,
rotas.icao_aerodromo_origem AS icao_origem, origem.name AS origem_name, origem.state AS origem_state, 
rotas.icao_aerodromo_destino AS icao_destino, destino.name AS destino_name,destino.state AS destino_state 
FROM (SELECT icao_empresa_aerea, icao_aerodromo_origem, icao_aerodromo_destino,frequencia, 
Rank() OVER (PARTITION BY icao_empresa_aerea ORDER BY frequencia DESC) AS rank 
FROM (SELECT icao_empresa_aerea, icao_aerodromo_origem, icao_aerodromo_destino, count(*) AS frequencia FROM vra GROUP BY 1,2,3)) rotas
LEFT JOIN airport origem
ON rotas.icao_aerodromo_origem = origem.icao
LEFT JOIN airport destino
ON rotas.icao_aerodromo_destino = destino.icao
INNER JOIN air_cia cia
ON rotas.icao_empresa_aerea = cia.icao
WHERE rank =1
GROUP BY razao_social,rotas.icao_empresa_aerea,origem_name,origem_state,destino_name,destino_state;
"""

# Execute o comando SQL para criar a view
cursor.execute(query)

# Certifique-se de confirmar as alterações e fechar a conexão
conn.commit()
conn.close()


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\gusta\AppData\Local\Temp\ipykernel_27200\2995110894.py:2: SyntaxWarning: invalid escape sequence '\d'
  conn = sqlite3.connect('.\data\data.db')


**Second view**

In [ ]:
# Conecte-se ao banco de dados SQLite (ou crie um novo)
conn = sqlite3.connect('.\data\data.db')

# Crie um cursor para executar comandos SQL
cursor = conn.cursor()

# Defina o comando SQL para criar uma view
query = """
CREATE VIEW top_company_per_airport AS

"""

# Execute o comando SQL para criar a view
cursor.execute(query)

# Certifique-se de confirmar as alterações e fechar a conexão
conn.commit()
conn.close()
